In [1]:
import Pkg
Pkg.add("SimpleCarModels")
Pkg.add("StaticArrays")
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
import FiniteDiff
import ForwardDiff
import Convex as cvx 
import ECOS
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
import MeshCat as mc 
import SimpleCarModels as scm
using StaticArrays

In [ ]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils","cartpole_animation.jl"))

**NOTE: This question will have long outputs for each cell, remember you can use `cell -> all output -> toggle scrolling` to better see it all**

## Q1: Direct Collocation (DIRCOL) for a Cart Pole (30 pts)

We are now going to start working with the NonLinear Program (NLP) Solver IPOPT to solve some trajectory optimization problems. First we will demonstrate how this works for simple optimization problems (not trajectory optimization). The interface that we have setup for IPOPT is the following:

$$ \begin{align} \min_{x} \quad & \ell(x) & \text{cost function}\\ 
 \text{st} \quad & c_{eq}(x) = 0 & \text{equality constraint}\\
 & c_L \leq c_{ineq}(x) \leq c_U & \text{inequality constraint}\\
 & x_L \leq x \leq x_U & \text{primal bound constraint}
 \end{align}$$
 
 where $\ell(x)$ is our objective function, $c_{eq}(x) = 0$ is our equality constraint, $c_L \leq c_{ineq}(x) \leq c_U$ is our bound inequality constraint, and $x_L \leq x \leq x_U $ is a bound constraint on our primal variable $x$. 
<!-- $$ \begin{align} \min_{x_{1:N},u_{1:N-1}} \quad & \sum_{i=1}^{N-1} \bigg[ \frac{1}{2} x_i^TQx_i + \frac{1}{2} u_i^TRu_i \bigg] + \frac{1}{2}x_N^TQ_fx_N\\ 
 \text{st} \quad & x_1 = x_{\text{IC}} \\ 
 & x_{i+1} = A x_i + Bu_i \quad \text{for } i = 1,2,\ldots,N-1 
 \end{align}$$ -->

## Part A: Solve an LP with IPOPT (5 pts)
To demonstrate this, we are going to ask you to solve a simple Linear Program (LP):
 
 $$ \begin{align} \min_{x} \quad & q^Tx\\ 
 \text{st} \quad & Ax = b \\
 & Gx \leq h
 \end{align}$$
 
 Your job will be to transform this problem into the form shown above and solve it with IPOPT.  To help you interface with IPOPT, we have created a function `fmincon` for you. Below is the docstring for this function that details all of the inputs. 

In [ ]:
"""
x = fmincon(cost,equality_constraint,inequality_constraint,x_l,x_u,c_l,c_u,x0,params,diff_type)

This function uses IPOPT to minimize an objective function 

`cost(params, x)` 

With the following three constraints: 

`equality_constraint(params, x) = 0`
`c_l <= inequality_constraint(params, x) <= c_u` 
`x_l <= x <= x_u` 

Note that the constraint functions should return vectors. 

Problem specific parameters should be loaded into params::NamedTuple (things like 
cost weights, dynamics parameters, etc.). 

args:
    cost::Function                    - objective function to be minimzed (returns scalar)
    equality_constraint::Function     - c_eq(params, x) == 0 
    inequality_constraint::Function   - c_l <= c_ineq(params, x) <= c_u 
    x_l::Vector                       - x_l <= x <= x_u 
    x_u::Vector                       - x_l <= x <= x_u 
    c_l::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    c_u::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    x0::Vector                        - initial guess 
    params::NamedTuple                - problem parameters for use in costs/constraints 
    diff_type::Symbol                 - :auto for ForwardDiff, :finite for FiniteDiff 
    verbose::Bool                     - true for IPOPT output, false for nothing 

optional args:
    tol                               - optimality tolerance 
    c_tol                             - constraint violation tolerance 
    max_iters                         - max iterations 
    verbose                           - verbosity of IPOPT 

outputs:
    x::Vector                         - solution 

You should try and use :auto for your `diff_type` first, and only use :finite if you 
absolutely cannot get ForwardDiff to work. 

This function will run a few basic checks before sending the problem off to IPOPT to 
solve. The outputs of these checks will be reported as the following:

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

If you're getting stuck during the testing of one of the derivatives, try switching 
to FiniteDiff.jl by setting diff_type = :finite. 
""";

In [ ]:
# initial_state = scm.SE2State(0, 0, pi/2)
# goal_state = scm.SE2State(5, 0, pi/2)

# Tractor-Trailer Parameters
# g_tractor_wheelbase = 3.0
# g_tractor_hitch_offset = 0.5
# g_trailer_wheelbase = 6.0
# g_velocity = 0.5
# max_alpha = atan(g_tractor_wheelbase/sqrt((g_trailer_wheelbase^2) - (g_tractor_hitch_offset^2)))
# q_current = [0,0,pi/2, pi]
# r1 = g_tractor_wheelbase/tan(max_alpha);
# psi = -1*(max_alpha/abs(max_alpha))*atan(g_tractor_hitch_offset/abs(r1));
# va = (g_velocity/abs(g_velocity))*abs(((g_velocity*g_tractor_hitch_offset*tan(max_alpha))/(g_tractor_wheelbase*sin(psi))));
# vb = va*abs(cos(psi-q_current[4]));
# r2 = g_tractor_hitch_offset/sin(psi);
# max_turn_radius_eq = g_trailer_wheelbase/sin(psi-q_current[4]);

# print("Max turning radius from straight ahead to equillibrium is: ", max_turn_radius_eq)
# initial_state

# states = scm.reedsshepp_waypoints(initial_state, goal_state, 51, r=max_turn_radius_eq)
# traj_length = scm.reedsshepp_length(initial_state, goal_state)

# println("REEDS-SHEPP States: ")
# println(states)
# println("Length of optimal trajectory: ", length)

# for i=1:size(states)
    
# end

In [2]:
# Function to get the REEDS-SHEPP warm start guess
function warm_start(initial, final, N, max_turn_radius)
    initial_state = SA_F64[initial[1],initial[2], initial[3]]
    goal_state = SA_F64[final[1], final[2], final[3]]
    
    reeds_shepp_traj = scm.reedsshepp_waypoints(initial_state, goal_state, N, r=max_turn_radius)
    reeds_shepp_length = scm.reedsshepp_length(initial_state, goal_state)
    
    return reeds_shepp_traj, reeds_shepp_length
end

function interpolate_states(params, initial_state, final_state, traj, rs_length)
    N = params.N
    dt = params.dt
    
#     interpolated_trajectory = range(initial_state, final_state, length = N)
    interpolated_trajectory = [zeros(6,) for y in traj]
    interpolated_trajectory[1] .= initial_state
    interpolated_trajectory[N] .= final_state
    
    average_speed = rs_length/((N-1)*dt)
    

    
    for i=2:N-1
        m = tan(traj[i-1][3] - pi/2)
        b = traj[i-1][2] - m*traj[i-1][1]
        weights = [-1*m, 1, -1*b]
        st = [traj[i][1], traj[i][2], 1]
        
        if sign(weights'*st)==+1
            interpolated_trajectory[i] .= [traj[i][1], traj[i][2], traj[i][3], pi, 0, average_speed]
        else
            interpolated_trajectory[i] .= [traj[i][1], traj[i][2], traj[i][3], pi, 0, -1*average_speed]
        end
        
        interpolated_trajectory[i] .= [traj[i][1], traj[i][2], traj[i][3], pi, 0, 0]
    end
    
    return interpolated_trajectory
end


# Testing the warm-start function
# initial_p = [0,0,pi/2,pi,0,0]
# goal_p = [0,5,pi/2,pi,0,0]
# max_turn_r = 72
# N = 51
# dt = 0.05
# tf = 2.5
# t_vec = 0:dt:tf 
# traj = warm_start(initial_p, goal_p, N, max_turn_r)
# println(traj[1][3])
# plot([x[1] for x in traj],[y[2] for y in traj])
# interpolated_traj = interpolate_states(N, initial_p, goal_p, traj);
# plot([x[1] for x in interpolated_traj],[y[2] for y in interpolated_traj])
# plot(t_vec, [x[3] for x in interpolated_traj])

interpolate_states (generic function with 1 method)

In [ ]:
function get_max_turn_radius(tractor_wheelbase, tractor_hitch_offset, trailer_wheelbase, nominal_velocity, max_alpha, initial_pose)
    
#     tractor_wheelbase, tractor_hitch_offset, trailer_wheelbase = params.tractor_wheelbase, params.tractor_hitch_offset, params.trailer_wheelbase
#     nominal_velocity, max_alpha = params.nominal_velocity, params.max_alpha
#     initial_pose = params.initial_pose
    
    r1 = tractor_wheelbase/tan(max_alpha);
    psi = -1*(max_alpha/abs(max_alpha))*atan(tractor_hitch_offset/abs(r1));
    va = (nominal_velocity/abs(nominal_velocity))*abs(((nominal_velocity*tractor_hitch_offset*tan(max_alpha))/(tractor_wheelbase*sin(psi))));
    vb = va*abs(cos(psi-initial_pose[4]));
    r2 = tractor_hitch_offset/sin(psi);
    max_turn_radius_eq = trailer_wheelbase/sin(psi-initial_pose[4]);
    
    return max_turn_radius_eq
    
end

function tractor_trailer_qdot(params::NamedTuple, q_current, u)
    
    tractor_wheelbase, tractor_hitch_offset, trailer_wheelbase = params.tractor_wheelbase, params.tractor_hitch_offset, params.trailer_wheelbase
    
    # Variable to store the rate of change of state
    alpha = 1*q_current[5]
    gear = 1
    velocity = 1*q_current[6]
    
    if(alpha!=0)
        r1 = tractor_wheelbase/tan(alpha);
        psi = -1*(alpha/abs(alpha))*atan(tractor_hitch_offset/abs(r1));
        if velocity==0
            va = 0
        else
            va = (velocity/abs(velocity))*abs(((gear*velocity*tractor_hitch_offset*tan(alpha))/(tractor_wheelbase*sin(psi))));
        end
        vb = va*abs(cos(psi-q_current[4]));
        r2 = tractor_hitch_offset/sin(psi);
        r3 = trailer_wheelbase/sin(psi-q_current[4]);
        omega_1 = gear*velocity*(1/r1);
        omega_2 = gear*velocity*((r2)/(r1*r3));
        trailer_theta_dot = omega_2;
        beta_dot = gear*velocity*(((r2)/(r1*r3)) - (1/r1));
    end

    # Special case when alpha equals zero
    if(alpha==0)
        
        va = gear*velocity;
        psi = 0;
        phi = (pi/2) - (abs(q_current[4]) - (pi/2));
        if(sin(phi)==0)
            beta_dot = 0;
            trailer_theta_dot = 0;
            vb = va*abs(cos(psi-q_current[4]));
        else
            hitch_turn_radius_zero_alpha = trailer_wheelbase/sin(phi);
            trailer_theta_dot = va/hitch_turn_radius_zero_alpha;
            beta_dot = trailer_theta_dot;
            vb = va*abs(cos(psi-q_current[4]));
        end
        
    end
    

    q_dot = [vb*cos(q_current[3]); vb*sin(q_current[3]); trailer_theta_dot; beta_dot; u[1]; u[2]]
    
    return q_dot;
    
end

function wrap_angles(x)
    new_state = 1*x
    
    if new_state[3]>(2*pi)
        new_state[3] = new_state[3]%(2*pi)
    elseif new_state[3]<0
        new_state[3] = (2*pi) - (new_state[3]%(2*pi)) 
    end
    
    if new_state[4]>(2*pi)
        new_state[4] = new_state[3]%(2*pi)
    elseif new_state[4]<0
        new_state[4] = (2*pi) - (new_state[3]%(2*pi)) 
    end
    
    return new_state
end

# function dynamics_rk4(params, x, u)
#     # RK4 integration with zero-order hold on u
#     h = params.dt
#     f1 = tractor_trailer_qdot(params, x, u)
#     f2 = tractor_trailer_qdot(params, x + 0.5*h*f1, u)
#     f3 = tractor_trailer_qdot(params, x + 0.5*h*f2, u)
#     f4 = tractor_trailer_qdot(params, x + h*f3, u)
    
#     next_state = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
# #     @time
#     # Wrap around for the orientation and cart angles
# #     return wrap_angles(next_state)
#     return next_state
    
# #     return x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
# end

function hermite_simpson(params::NamedTuple, x1::Vector, x2::Vector, u, dt::Real)::Vector
    # TODO: input hermite simpson implicit integrator residual 
    x1dot = tractor_trailer_qdot(params, x1, u)
    x2dot = tractor_trailer_qdot(params, x2, u)
    x1_2 = 0.5*(x1 + x2) + (dt/8)*(x1dot - x2dot)
    x1_2dot = tractor_trailer_qdot(params, x1_2, u)
    return x1 + (dt/6)*(x1dot + 4*x1_2dot + x2dot) - x2
end

To solve this problem with IPOPT and `fmincon`, we are going to concatenate all of our $x$'s and $u$'s into one vector:

$$ Z = \begin{bmatrix}x_1 \\ u_1 \\ x_2 \\ u_2 \\ \vdots \\ x_{N-1} \\ u_{N-1} \\ x_N \end{bmatrix} \in \mathbb{R}^{N \cdot nx + (N-1)\cdot nu} $$ 

where $x \in \mathbb{R}^{nx}$ and $u \in \mathbb{R}^{nu}$. Below we will provide useful indexing guide in `create_idx` to help you deal with $Z$.

It is also worth noting that while there are inequality constraints present ($-10 \leq u_i \leq 10$), we do not need a specific `inequality_constraints` function as an input to `fmincon` since these are just bounds on the primal ($Z$) variable. You should use primal bounds in `fmincon` to capture these constraints.  

In [ ]:
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

function trajectory_cost(params::NamedTuple, Z::Vector)::Real
    idx, N, xg = params.idx, params.N, params.goal_pose
    Q, R, Qf = params.Q, params.R, params.Qf
    
    J = 0 
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
       
        # The stage cost penalizes the states based on how far away they are from the goal
        J += 0.5*(xi - xg)[1:3]'*Q[1:3,1:3]*(xi - xg)[1:3] + 0.5*ui'*R*ui

    end
    
    # dont forget terminal cost 
    xN = Z[idx.x[N]]
    J += 0.5*(xN-xg)[1:3]'*Qf[1:3,1:3]*(xN-xg)[1:3]
    
    return J 
end

function tt_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    
    # TODO: create dynamics constraints using hermite simpson 
    
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        c[idx.c[i]] = hermite_simpson(params, xi, xip1, ui, dt)
    end
    
#     for i = 1:(N-1)
#         xi = Z[idx.x[i]]
#         ui = Z[idx.u[i]] 
# #         xip1 = Z[idx.x[i+1]]
        
#         c[idx.c[i]] .= Z[idx.x[i+1]] - dynamics_rk4(params, xi, ui) # - xip1
#     end
#     println("Size of only dynamics constraints: ", size(c))
    return c 
#     @time
end

function tt_equality_constraint(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic, xg = params.N, params.idx, params.initial_pose, params.goal_pose 
    
    # TODO: return all of the equality constraints 
    eq_initial = Z[idx.x[1]] - xic
    eq_final = Z[idx.x[N]] - xg
    
    return vcat(eq_initial, tt_dynamics_constraints(params, Z), eq_final[1:3])
#     @time
    
end

In [ ]:
# Test the reeds-shepp curve generator and trajectory interpolation
# function solve_tt_trajectory
#     tractor_wheelbase = 3.0
#     tractor_hitch_offset = 0.5
#     trailer_wheelbase = 6.0
#     max_alpha = atan(tractor_wheelbase/sqrt((trailer_wheelbase^2) - (tractor_hitch_offset^2)))
#     nominal_velocity = 0.5;
#     # println(tan(max_alpha))
    
#     # Initial Pose and Goal Pose
#     initial_pose = [0,0,pi/2,pi,0,0]
#     goal_pose = [0,5,pi/2,pi,0,0]
    
#     max_turn_radius = get_max_turn_radius(tractor_wheelbase, tractor_hitch_offset, trailer_wheelbase, nominal_velocity, max_alpha, initial_pose)
    
    
# end
dt = 0.05
tf = 5.0
t_vec = 0:dt:tf 
N = length(t_vec)
i_pose = [0,0,pi/2,pi,0,0]
g_pose = [0,5,pi/2,pi,0,0]
test_x_traj_guess = range(i_pose, g_pose, length = N)
# # plot([x[1] for x in test_x_traj_guess],[y[2] for y in test_x_traj_guess])
# display(plot!(t_vec, test_x_traj_guess', label = ["|r_1 - r_2|" "|r_1 - r_3|" "|r_2 - r_2|"]))
# plot(t_vec, [x[3] for x in test_x_traj_guess], lims=[0,4*pi])
# plot(t_vec, [x[4] for x in test_x_traj_guess], lims=[0,4*pi])

In [1]:
function solve_tt_trajectory(;verbose=true)
    
    # problem size 
    nx = 6
    nu = 2
    dt = 0.05
    tf = 30.0
    t_vec = 0:dt:tf 
    N = length(t_vec)
    
    
    # Tractor-Trailer Parameters
    tractor_wheelbase = 3.0
    tractor_hitch_offset = 0.5
    trailer_wheelbase = 6.0
    max_alpha = atan(tractor_wheelbase/sqrt((trailer_wheelbase^2) - (tractor_hitch_offset^2)))
    nominal_velocity = 0.5
    
    # Initial Pose and Goal Pose
    initial_pose = [2,1,pi/2,pi,0,0]
    goal_pose = [20,1,pi/2,pi,0,0]
    
    println("Starting the DIRCOL solve!")
    println("Initial State: ", initial_pose)
    println("Final State: ", goal_pose)
    
    # Get the maximum turning radius for the REEDS-SHEPP Generator
    max_turn_radius = get_max_turn_radius(tractor_wheelbase, tractor_hitch_offset, trailer_wheelbase, nominal_velocity, max_alpha, initial_pose)
    println("Max turn radius: ", max_turn_radius)
    
    # LQR cost 
    Q = diagm(ones(nx))
    R = 0.1*diagm(ones(nu))
    Qf = 10*diagm(ones(nx))
    
    # indexing 
    idx = create_idx(nx,nu,N)
    
    # load all useful things into params 
    params = (Q = Q,
        Qf = Qf,
        R = R,
        tractor_wheelbase = tractor_wheelbase, 
        tractor_hitch_offset = tractor_hitch_offset, 
        trailer_wheelbase = trailer_wheelbase,
        nominal_velocity = 0.5,
        max_alpha = max_alpha,
        initial_pose = initial_pose,
        goal_pose = goal_pose,
        N = N,
        idx = idx,
        max_turn_radius = max_turn_radius,
        dt = dt)
    
    # Constraints on the primal variable
    x_l = -1*Inf*ones(idx.nz)
    x_u = Inf*ones(idx.nz)

    
    for i = 1:(N-1)
        
        # Constraints on the trailer orientation in the global frame 
        x_l[idx.x[i][3]] = 0
        x_u[idx.x[i][3]] = 2*pi

        # Constraints on the tractor-trailer angle
        x_l[idx.x[i][4]] = 0
        x_u[idx.x[i][4]] = 2*pi

        # Constraint on max steering angle
        x_l[idx.x[i][5]] = -1*max_alpha
        x_u[idx.x[i][5]] = 1*max_alpha
        
        # Constraint on max velocity
        x_l[idx.x[i][6]] = -15
        x_u[idx.x[i][6]] = 15
        
        # Constraint on steering rate
        x_l[idx.u[i][1]] = -1*(max_alpha/3)
        x_u[idx.u[i][1]] = 1*(max_alpha/3)
        
        # Constraint on acceleration
#         x_l[idx.u[i][2]] = -2.5
#         x_u[idx.u[i][2]] = 2.5
    end
    
#     println("Max element of x_l: ", maximum(x_l))
#     println("Min element of x_u: ", minimum(x_u))
#     println("Value of max alpha: ", max_alpha/3)
    
    # inequality constraint bounds (this is what we do when we have no inequality constraints)
    c_l = zeros(0)
    c_u = zeros(0)
    function inequality_constraint(params, Z)
        return zeros(eltype(Z), 0)
    end 
    
    # Get the initial trajectory guess
    init_traj, rs_length = warm_start(initial_pose, goal_pose, N, max_turn_radius) # Warm start the solver using a REEDS-SHEPP curve
    interpolated_trajectory = interpolate_states(params, initial_pose, goal_pose, init_traj, rs_length)
    
#     plot([x[1] for x in interpolated_trajectory],[y[2] for y in interpolated_trajectory])
    println("warm start")
    z0 = 0.001*randn(idx.nz)
    test_x_traj_guess = range(initial_pose, goal_pose, length = N)
    for i=1:N-1
#         z0[idx.x[i]] += 1*test_x_traj_guess[i]
#         z0[idx.x[i][5]] = 0.001
#         z0[idx.x[i][6]] = 0.001
#         z0[idx.u[i][1]] = 0.001
#         z0[idx.u[i][2]] = 1
        
        z0[idx.x[i]] .+= 1*interpolated_trajectory[i]
    end
    z0[idx.x[N]] .+= 1*interpolated_trajectory[N]
#     z0[idx.x[N]] += 1*test_x_traj_guess[N]
    
    # Test equality constraint count
    equality_cons = tt_equality_constraint(params, z0)
    println("Size of equality constraints: ", size(equality_cons))
    
    # choose diff type (try :auto, then use :finite if :auto doesn't work)
    diff_type = :auto 
#     diff_type = :finite
    
    # Solve the minimization problem
    Z = fmincon(trajectory_cost,tt_equality_constraint,inequality_constraint,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    
    return X, U, t_vec, params 
end

X, U, t_vec, p = solve_tt_trajectory(verbose=true)
# display(plot([x[1] for x in X],[y[2] for y in X]))
for i=1:N
    println(X[i])
end
# display(plot(t_vec[1:p.N-1], [u[1] for u in U]))
# display(plot(t_vec[1:p.N-1], [u[2] for u in U]))

Starting the DIRCOL solve!
Initial State: [1.0, 1.0, 1.5707963267948966, 3.141592653589793, 0.0, 0.0]
Final State: [3.0, 1.0, 1.5707963267948966, 3.141592653589793, 0.0, 0.0]


LoadError: UndefVarError: get_max_turn_radius not defined